# Using Pipeline class from Transformers Library


In [ ]:
import pathlib
from transformers import pipeline
from PIL import Image
import matplotlib.pylab as plt
import numpy as np

#### Extract all the images from our dataset and divide them into ground truths and unsegmented images.

In [ ]:
imagepath = pathlib.Path('./Images/')
images = imagepath.glob('*')
inferred = []
GT = []
for image in images:
#     print(dir(image))
    
    if "GT" in image.name:
        GT.append(image)
#         print("GT" , image.name)
 
    elif "GT" not in image.name and "ipy" not in image.name:
        inferred.append(image)
#         print("inf", image.name)
GT_paths = sorted(GT)
input_paths = sorted(inferred)




# Test the generated mask with ground truth using SEE-Segment

In [ ]:
import sys
sys.path.append('./see-segment/')

In [ ]:
from see import Segment_Fitness as sf

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

models = ["facebook/mask2former-swin-large-cityscapes-semantic",
          "nvidia/segformer-b0-finetuned-cityscapes-1024-1024",
          "apple/deeplabv3-mobilevit-small"]
#           "facebook/mask2former-swin-large-mapillary-vistas-semantic"]
#           "Intel/dpt-large-ade"]
row = []
GT_images = []
input_images = []
each_row = []
for image_path, GT_path in zip(input_paths[0:1], GT_paths[0:1]):
    image = Image.open(image_path).convert("RGB")
    input_images.append(image)
    
    GT_image = Image.open(GT_path)
    GT_images.append(GT_image)
    
    
    # Create subplots for each model's segmented image
    fig, axs = plt.subplots(1, len(models))

    # Iterate over the models and display segmented images
    for i, model_name in enumerate(models):
        # Initialize an image segmentation pipeline
        segmentation_pipeline = pipeline("image-segmentation", model=model_name)
        outputs = segmentation_pipeline(image)
        segmented_image = outputs[0]['mask']
        each_row.append(segmented_image)
        # Display the segmented image in the corresponding subplot
        ax = axs[i] if len(models) > 1 else axs
        ax.imshow(segmented_image)
#         ax.axis('off')
    row.append(each_row)
    each_row = []
    # Adjust the layout to avoid overlapping images
    fig.tight_layout()

    # Show the figure for the current image
    plt.show()


## Problem
When using the fitness function, I get an Assertion Error that says that the sizes of the two images I am trying to comapre are different. I checked the shapes and for some reason shape of all ground truth images is nothing. I am not sure why that is?

In [ ]:
np.array(GT_images[0]).shape

In [ ]:

print(np.array(row[0][0]).shape)

In [ ]:
for counter in range(0,len(inferred)):
    for i in range(0,len(models)):
        sf.FitnessFunction(np.array(GT_images[counter]), np.array(row[counter][i]))